# Stochastic Partial Derivative Equations

<!-- SUMMARY: Estimation and Simulations performed in the framework of SPDE -->

<!-- CATEGORY: SPDE -->

In this tutorial, we show how to use the SPDE. We compare different calculations obtained through gstlearn API interfaces, either utilizing Cholesky decomposition or Matrix-free calculations.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import matplotlib.pyplot as plt

from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np

gdoc.setNoScroll()

## Parameters

In [ ]:
# Data
np.random.seed(123)
ndat = 1000

# Model
rangev = 0.2
sill = 1.
nugget = 0.1

# Grid 
nx = [50,50]
dx = [0.02,0.02]
x0 = [0,0]

#Grid meshing
nxm = [75,75]
dxm = [0.02,0.02]
x0m = [-0.25,-0.25]

dbfmt = gl.DbStringFormat.createFromFlags(flag_stats=True, names=["spde*"])

### Grid and Meshing

In [ ]:
grid = gl.DbGrid.create(nx,dx,x0)
gridExt = gl.DbGrid.create(nxm,dxm,x0m)
mesh = gl.MeshETurbo(gridExt)

### Model

In [ ]:
model = gl.Model.createFromParam(gl.ECov.MATERN,param=1,range=rangev,sill=sill)
model.addCovFromParam(gl.ECov.NUGGET,sill=nugget)
model.setDriftIRF()

In [ ]:
model

### Data

In [ ]:
dat = gl.Db.create()
dat["x"] = np.random.uniform(size=ndat)
dat["y"] = np.random.uniform(size=ndat)
dat.setLocators(["x","y"],gl.ELoc.X)

## SPDE non-conditional simulation

### Grid query

In [ ]:
gl.law_set_random_seed(131351)
gl.simulateSPDE(None, grid, model)
grid.display(dbfmt)

In [ ]:
gp.init(figsize=[7,7],flagEqual=True)
gp.raster(grid)
gp.decoration(title="Non Conditional Simulation")

### Data query

In [ ]:
gl.law_set_random_seed(131351)
gl.simulateSPDE(None, dat, model)
dat.display(dbfmt)

## Kriging

In [ ]:
gl.krigingSPDE(dat, grid, model, useCholesky=True, namconv=gl.NamingConvention("KrigingSPDE-Chol"))
gl.krigingSPDE(dat, grid, model, useCholesky=False, namconv=gl.NamingConvention("KrigingSPDE-Free"))
grid.display(dbfmt)

In [ ]:
gp.init(figsize=[7,7],flagEqual=True)
gp.raster(grid)
gp.symbol(dat, c='black')
gp.decoration(title="Estimation")

## Likelihood

- Using the new API (we use the same 'mesh' as for the manual case to obtain the same results).

In [ ]:
nMC = 100
params = gl.SPDEParam.create(nMC=nMC)
loglike = gl.logLikelihoodSPDE(dat, model, useCholesky=1, meshes=[mesh], params=params, verbose=False)
print(f"-> logLikelihood with cholesky=1 {round(loglike,4)}")

In [ ]:
nMC = 100
params = gl.SPDEParam.create(nMC=nMC)
loglike = gl.logLikelihoodSPDE(dat, model, useCholesky=0, meshes=[mesh], params=params, verbose=False)
print(f"-> logLikelihood with cholesky=0 {round(loglike,4)}")